In [1]:
import json
import pandas as pd

In [61]:
inpath = '/Users/dongmeichen/OneDrive - lanecouncilofgovernments/VE-RSPM'

In [62]:
scen = pd.read_csv(f'{inpath}/sensitivity_tests/scenario_list.csv')

In [63]:
scen.head()

,B,C,D,E,F,G,I,P,T,V,S
0,0,1,1,1,0,1,1,0,1,1,B0C1D1E1F0G1I1P0T1V1
1,1,1,1,1,0,1,1,0,1,1,B1C1D1E1F0G1I1P0T1V1
2,2,1,1,1,0,1,1,0,1,1,B2C1D1E1F0G1I1P0T1V1
3,0,2,1,1,0,1,1,0,1,1,B0C2D1E1F0G1I1P0T1V1
4,1,2,1,1,0,1,1,0,1,1,B1C2D1E1F0G1I1P0T1V1


In [64]:
scen = scen.rename(columns={'S': 'Scenario'})
#scen.rename(columns={'S': 'Scenario'}, inplace=True)
scen.head()

,B,C,D,E,F,G,I,P,T,V,Scenario
0,0,1,1,1,0,1,1,0,1,1,B0C1D1E1F0G1I1P0T1V1
1,1,1,1,1,0,1,1,0,1,1,B1C1D1E1F0G1I1P0T1V1
2,2,1,1,1,0,1,1,0,1,1,B2C1D1E1F0G1I1P0T1V1
3,0,2,1,1,0,1,1,0,1,1,B0C2D1E1F0G1I1P0T1V1
4,1,2,1,1,0,1,1,0,1,1,B1C2D1E1F0G1I1P0T1V1


In [5]:
meas = pd.read_csv(f'{inpath}/output/Measures_Sensitivity_Output_2.csv')
meas.head()

,Measure,Units,Description,Value2010,B0C1D1E1F0G1I1P0T1V1,B1C1D1E1F0G1I1P0T1V1,B2C1D1E1F0G1I1P0T1V1,B0C2D1E1F0G1I1P0T1V1,B1C2D1E1F0G1I1P0T1V1
0,HhTotDailyOthParkingCost,USD per day,Total daily non-work parking expenditures by h...,28241.275835,10480.754212,10480.754212,10480.754212,10480.754212,10480.754212
1,HhTotDailyWkrParkingCost,USD per day,Total daily work parking expenditures by house...,37225.367097,11550.788950,11550.788950,11550.788950,11550.788950,11550.788950
2,HhVehTravCostShare,Proportion,Household vehicle Own & out-of-pocket costs sh...,0.171750,0.044480,0.044178,0.043809,0.044701,0.044397
3,HhVehTravCostShareLowInc,Proportion,Low income (0to20K 2010$) household vehicle Ow...,0.312967,0.337755,0.337755,0.337754,0.341369,0.341369
4,MareaCarSvcAutoDvmtProp,Proportion,Average proportion car service vehicle DVMT in...,1.000000,0.983049,0.983047,0.983043,0.981926,0.981928


In [68]:
for x in scen.iterrows():
    print(x)
    break

(0, B                              0
C                              1
D                              1
E                              1
F                              0
G                              1
I                              1
P                              0
T                              1
V                              1
Scenario    B0C1D1E1F0G1I1P0T1V1
Name: 0, dtype: object)


In [65]:
def rows_to_dicts(df):
    out = []
    fields = list(df)
    out = [{field:val for field, val in zip(fields, values)} 
           for _, values in df.iterrows()]
    return out

In [108]:
fields = list(scen)

In [109]:
fields

['B', 'C', 'D', 'E', 'F', 'G', 'I', 'P', 'T', 'V', 'Scenario']

In [66]:
output = rows_to_dicts(scen)

In [69]:
print(len(output))
len(scen.Scenario.unique())

69984


69984

In [70]:
co2_vals = [x for x in meas['Measure'] if 'CO2' in x]
co2_vals

['UrbanBusCO2e',
 'UrbanBusCO2eRate',
 'UrbanComSvcCO2e',
 'UrbanHhCO2e',
 'UrbanHvyTrkAveCO2eRate',
 'UrbanHvyTrkCO2e',
 'UrbanLdvCO2e',
 'UrbanLdvCO2eRate',
 'UrbanRailCO2e',
 'UrbanVanCO2e']

In [71]:
len(co2_vals)

10

In [72]:
def vals_from_scenario(scenario, meas_df, subset=None):
    subset = meas_df.Measure.unique() if subset is None else subset
    try:
        out = {field: meas_df.loc[meas_df.Measure == field, scenario].values[0]
              for field in subset}
        return out
    except KeyError:
        return {}
    except:
        raise

In [73]:
vals_from_scenario('B0C1D1E1F0G1I1P0T1V1', meas, co2_vals)

{'UrbanBusCO2e': 6469.76823895145,
 'UrbanBusCO2eRate': 1231.55839744099,
 'UrbanComSvcCO2e': 34237.365820358,
 'UrbanHhCO2e': 458407.02164538,
 'UrbanHvyTrkAveCO2eRate': 855.711475785546,
 'UrbanHvyTrkCO2e': 633054.431485929,
 'UrbanLdvCO2e': 500437.687112108,
 'UrbanLdvCO2eRate': 0.00016033916729636498,
 'UrbanRailCO2e': 0.0,
 'UrbanVanCO2e': 7793.299646370079}

In [74]:
def append_scenario_vals(outputs, meas_df, subset=None):
    for d in outputs:
        scenario = d['Scenario']
        scenario_vals = vals_from_scenario(scenario, meas_df, subset)
        d.update(scenario_vals)
    return outputs

In [75]:
test = append_scenario_vals(output, meas, co2_vals)

In [76]:
test[0]

{'B': 0,
 'C': 1,
 'D': 1,
 'E': 1,
 'F': 0,
 'G': 1,
 'I': 1,
 'P': 0,
 'T': 1,
 'V': 1,
 'Scenario': 'B0C1D1E1F0G1I1P0T1V1',
 'UrbanBusCO2e': 6469.76823895145,
 'UrbanBusCO2eRate': 1231.55839744099,
 'UrbanComSvcCO2e': 34237.365820358,
 'UrbanHhCO2e': 458407.02164538,
 'UrbanHvyTrkAveCO2eRate': 855.711475785546,
 'UrbanHvyTrkCO2e': 633054.431485929,
 'UrbanLdvCO2e': 500437.687112108,
 'UrbanLdvCO2eRate': 0.00016033916729636498,
 'UrbanRailCO2e': 0.0,
 'UrbanVanCO2e': 7793.299646370079}

In [77]:
test[-1]

{'B': 2,
 'C': 3,
 'D': 3,
 'E': 3,
 'F': 3,
 'G': 3,
 'I': 3,
 'P': 3,
 'T': 3,
 'V': 2,
 'Scenario': 'B2C3D3E3F3G3I3P3T3V2'}

In [78]:
with open(f'{inpath}/VEScenarioViewer/test_out.json', 'w') as f:
    json.dump(test, f)

In [38]:
#!open ./test_out.json

In [39]:
#!rm ./test_out.json

In [90]:
# organize measure notes
meas_notes = meas[["Measure", "Units", "Description"]]

In [91]:
meas_notes.head()

,Measure,Units,Description
0,HhTotDailyOthParkingCost,USD per day,Total daily non-work parking expenditures by h...
1,HhTotDailyWkrParkingCost,USD per day,Total daily work parking expenditures by house...
2,HhVehTravCostShare,Proportion,Household vehicle Own & out-of-pocket costs sh...
3,HhVehTravCostShareLowInc,Proportion,Low income (0to20K 2010$) household vehicle Ow...
4,MareaCarSvcAutoDvmtProp,Proportion,Average proportion car service vehicle DVMT in...


In [92]:
# NAME, LABEL, DESCRIPTION, INSTRUCTIONS, METRIC, UNIT, COLUMN
meas_notes = meas_notes.rename(columns={'Measure': 'NAME', 'Units': 'UNIT', 'Description':'DESCRIPTION'})

In [93]:
meas_notes.head()

,NAME,UNIT,DESCRIPTION
0,HhTotDailyOthParkingCost,USD per day,Total daily non-work parking expenditures by h...
1,HhTotDailyWkrParkingCost,USD per day,Total daily work parking expenditures by house...
2,HhVehTravCostShare,Proportion,Household vehicle Own & out-of-pocket costs sh...
3,HhVehTravCostShareLowInc,Proportion,Low income (0to20K 2010$) household vehicle Ow...
4,MareaCarSvcAutoDvmtProp,Proportion,Average proportion car service vehicle DVMT in...


In [99]:
meas_notes.loc[:,'LABEL'] = meas_notes.loc[:,'NAME']
meas_notes.loc[:,'COLUMN']= meas_notes.loc[:,'NAME']

In [100]:
meas_notes.head()

,NAME,UNIT,DESCRIPTION,LABEL,COLUMN
0,HhTotDailyOthParkingCost,USD per day,Total daily non-work parking expenditures by h...,HhTotDailyOthParkingCost,HhTotDailyOthParkingCost
1,HhTotDailyWkrParkingCost,USD per day,Total daily work parking expenditures by house...,HhTotDailyWkrParkingCost,HhTotDailyWkrParkingCost
2,HhVehTravCostShare,Proportion,Household vehicle Own & out-of-pocket costs sh...,HhVehTravCostShare,HhVehTravCostShare
3,HhVehTravCostShareLowInc,Proportion,Low income (0to20K 2010$) household vehicle Ow...,HhVehTravCostShareLowInc,HhVehTravCostShareLowInc
4,MareaCarSvcAutoDvmtProp,Proportion,Average proportion car service vehicle DVMT in...,MareaCarSvcAutoDvmtProp,MareaCarSvcAutoDvmtProp


In [101]:
meas_notes.loc[:,'INSTRUCTIONS'] = meas_notes.loc[:,'DESCRIPTION']

In [102]:
meas_notes.loc[:,'METRIC'] = 'Average'

In [103]:
meas_notes = meas_notes[['NAME', 'LABEL', 'DESCRIPTION', 'INSTRUCTIONS', 'METRIC', 'UNIT', 'COLUMN']]

In [104]:
meas_notes.head()

,NAME,LABEL,DESCRIPTION,INSTRUCTIONS,METRIC,UNIT,COLUMN
0,HhTotDailyOthParkingCost,HhTotDailyOthParkingCost,Total daily non-work parking expenditures by h...,Total daily non-work parking expenditures by h...,Average,USD per day,HhTotDailyOthParkingCost
1,HhTotDailyWkrParkingCost,HhTotDailyWkrParkingCost,Total daily work parking expenditures by house...,Total daily work parking expenditures by house...,Average,USD per day,HhTotDailyWkrParkingCost
2,HhVehTravCostShare,HhVehTravCostShare,Household vehicle Own & out-of-pocket costs sh...,Household vehicle Own & out-of-pocket costs sh...,Average,Proportion,HhVehTravCostShare
3,HhVehTravCostShareLowInc,HhVehTravCostShareLowInc,Low income (0to20K 2010$) household vehicle Ow...,Low income (0to20K 2010$) household vehicle Ow...,Average,Proportion,HhVehTravCostShareLowInc
4,MareaCarSvcAutoDvmtProp,MareaCarSvcAutoDvmtProp,Average proportion car service vehicle DVMT in...,Average proportion car service vehicle DVMT in...,Average,Proportion,MareaCarSvcAutoDvmtProp


In [117]:
meas_notes_subset = meas_notes.loc[meas_notes['NAME'].isin(co2_vals)]

In [118]:
output2 = rows_to_dicts(meas_notes_subset)

In [119]:
output2[0]

{'NAME': 'UrbanBusCO2e',
 'LABEL': 'UrbanBusCO2e',
 'DESCRIPTION': 'Average annual production of greenhouse gas emissions from public transit bus travel in the urban area',
 'INSTRUCTIONS': 'Average annual production of greenhouse gas emissions from public transit bus travel in the urban area',
 'METRIC': 'Average',
 'UNIT': 'Metric tons CO2e per year',
 'COLUMN': 'UrbanBusCO2e'}

In [120]:
with open(f'{inpath}/VEScenarioViewer/test_out2.json', 'w') as f:
    json.dump(output2, f)

In [122]:
# organize category and levels
cat_levels = pd.read_csv(f'{inpath}/sensitivity_tests/sensitivity_scenarios.csv')

In [123]:
cat_levels.head()

,strategy_label,strategy_description,strategy_level,category_name,category_label,category_description,category_instructions,file,geo,city,variable,policy_name,policy_label,policy_description,value
0,Community Design,Local policies to enable shorter trips and tra...,0,B,Bicycles,"Network improvements, incentives, and technolo...",The diversion of single-occupant vehicle trave...,azone_prop_sov_dvmt_diverted.csv,azone,NaN,PropSovDvmtDiverted,0,Reduced,Reduced bicyling percentage of SOV tours less ...,0.1
1,Community Design,Local policies to enable shorter trips and tra...,1,B,Bicycles,"Network improvements, incentives, and technolo...",The diversion of single-occupant vehicle trave...,azone_prop_sov_dvmt_diverted.csv,azone,NaN,PropSovDvmtDiverted,1,Base,Baseline bicyling percentage of SOV tours less...,0.19
2,Community Design,Local policies to enable shorter trips and tra...,2,B,Bicycles,"Network improvements, incentives, and technolo...",The diversion of single-occupant vehicle trave...,azone_prop_sov_dvmt_diverted.csv,azone,NaN,PropSovDvmtDiverted,2,Increased,Increased bicyling percentage of SOV tours les...,0.3
3,Community Design,Local policies to enable shorter trips and tra...,0,P,Parking,Extent of and charges for priced parking,The extent of paid parking and its price,bzone_parking.csv,bzone,NaN,PkgCost.2005,0,Reduced,Reduced average daily cost for long-term parki...,1.44
4,Community Design,Local policies to enable shorter trips and tra...,1,P,Parking,Extent of and charges for priced parking,The extent of paid parking and its price,bzone_parking.csv,bzone,NaN,PkgCost.2005,1,Base,Baseline average daily cost for long-term park...,2.88


In [124]:
list(cat_levels.columns)

['strategy_label',
 'strategy_description',
 'strategy_level',
 'category_name',
 'category_label',
 'category_description',
 'category_instructions',
 'file',
 'geo',
 'city',
 'variable',
 'policy_name',
 'policy_label',
 'policy_description',
 'value']

In [125]:
# NAME, LABEL, DESCRIPTION, INSTRUCTIONS, LEVELS (NAME, LABEL, DESCRIPTION)
levels = cat_levels[['policy_name', 'policy_label', 'policy_description']]

In [126]:
levels = levels.rename(columns={'policy_name': 'NAME', 'policy_label': 'LABEL', 'policy_description':'DESCRIPTION'})

In [127]:
levels.head()

,NAME,LABEL,DESCRIPTION
0,0,Reduced,Reduced bicyling percentage of SOV tours less ...
1,1,Base,Baseline bicyling percentage of SOV tours less...
2,2,Increased,Increased bicyling percentage of SOV tours les...
3,0,Reduced,Reduced average daily cost for long-term parki...
4,1,Base,Baseline average daily cost for long-term park...


In [128]:
level_output = rows_to_dicts(levels)
level_output[0]

{'NAME': 0,
 'LABEL': 'Reduced',
 'DESCRIPTION': 'Reduced bicyling percentage of SOV tours less than 20 miles'}

In [129]:
type(level_output[0])

dict

In [130]:
categories = cat_levels[['category_name','category_label','category_description','category_instructions']]
categories = categories.rename(columns={'category_name': 'NAME', 
                                        'category_label': 'LABEL', 
                                        'category_description':'DESCRIPTION',
                                        'category_instructions': 'INSTRUCTIONS'})

In [131]:
category_output = rows_to_dicts(categories)
category_output[0]

{'NAME': 'B',
 'LABEL': 'Bicycles',
 'DESCRIPTION': 'Network improvements, incentives, and technologies that encourage bicycling and other light-weight vehicle travel',
 'INSTRUCTIONS': 'The diversion of single-occupant vehicle travel to bicycles, electric bicycles and other light-weight vehicles'}

In [135]:
def list_levels_in_cat(cat, cat_levels_df):
    df = cat_levels_df.loc[cat_levels_df['category_name'] == cat]
    df = df[['policy_name', 'policy_label', 'policy_description']]
    df = df.rename(columns={'policy_name': 'NAME',
                            'policy_label': 'LABEL',
                            'policy_description':'DESCRIPTION'})
    out = rows_to_dicts(df)
    return out

In [151]:
def list_cateory_info(cat, cat_levels_df):
    df = cat_levels_df.loc[cat_levels_df['category_name'] == cat]
    df = df[['category_name','category_label','category_description','category_instructions']].drop_duplicates()
    df = df.rename(columns={'category_name': 'NAME', 
                            'category_label': 'LABEL', 
                            'category_description':'DESCRIPTION',
                            'category_instructions': 'INSTRUCTIONS'})
    out = rows_to_dicts(df)
    return out    

In [136]:
list_levels_in_cat('B', cat_levels)

[{'NAME': 0,
  'LABEL': 'Reduced',
  'DESCRIPTION': 'Reduced bicyling percentage of SOV tours less than 20 miles'},
 {'NAME': 1,
  'LABEL': 'Base',
  'DESCRIPTION': 'Baseline bicyling percentage of SOV tours less than 20 miles (19%)'},
 {'NAME': 2,
  'LABEL': 'Increased',
  'DESCRIPTION': 'Increased bicyling percentage of SOV tours less than 20 miles'}]

In [150]:
cat_levels[['category_name','category_label','category_description','category_instructions']].drop_duplicates().head()

,category_name,category_label,category_description,category_instructions
0,B,Bicycles,"Network improvements, incentives, and technolo...",The diversion of single-occupant vehicle trave...
3,P,Parking,Extent of and charges for priced parking,The extent of paid parking and its price
15,T,Transit,Level of public transit service,The extent and frequency of transit service
23,D,Demand Management,Programs to encourage less private vehicle travel,Programs and incentives which encourage people...
33,E,Driving Efficiency,Driving efficiency by increasing implementatio...,DVMT affected by operations for different road...


In [152]:
list_cateory_info('B', cat_levels)

[{'NAME': 'B',
  'LABEL': 'Bicycles',
  'DESCRIPTION': 'Network improvements, incentives, and technologies that encourage bicycling and other light-weight vehicle travel',
  'INSTRUCTIONS': 'The diversion of single-occupant vehicle travel to bicycles, electric bicycles and other light-weight vehicles'}]

In [162]:
def add_nested_dicts(dict1, dict2):
    dict1['LEVELS'] = dict2
    return dict1

In [171]:
def collect_scenario_info(cat_levels_df):
    out = []
    for cat in list(cat_levels_df.category_name.unique()):
        dict1 = list_cateory_info(cat, cat_levels_df)
        dict2 = list_levels_in_cat(cat, cat_levels_df)
        out.append(add_nested_dicts(dict1[0], dict2[0]))
    return out

In [158]:
dict1 = list_cateory_info('B', cat_levels)
dict2 = list_levels_in_cat('B', cat_levels)

In [160]:
dict1

[{'NAME': 'B',
  'LABEL': 'Bicycles',
  'DESCRIPTION': 'Network improvements, incentives, and technologies that encourage bicycling and other light-weight vehicle travel',
  'INSTRUCTIONS': 'The diversion of single-occupant vehicle travel to bicycles, electric bicycles and other light-weight vehicles'}]

In [161]:
dict2

[{'NAME': 0,
  'LABEL': 'Reduced',
  'DESCRIPTION': 'Reduced bicyling percentage of SOV tours less than 20 miles'},
 {'NAME': 1,
  'LABEL': 'Base',
  'DESCRIPTION': 'Baseline bicyling percentage of SOV tours less than 20 miles (19%)'},
 {'NAME': 2,
  'LABEL': 'Increased',
  'DESCRIPTION': 'Increased bicyling percentage of SOV tours less than 20 miles'}]

In [165]:
type(dict2)

list

In [166]:
add_nested_dicts(dict1[0], dict2[0])

{'NAME': 'B',
 'LABEL': 'Bicycles',
 'DESCRIPTION': 'Network improvements, incentives, and technologies that encourage bicycling and other light-weight vehicle travel',
 'INSTRUCTIONS': 'The diversion of single-occupant vehicle travel to bicycles, electric bicycles and other light-weight vehicles',
 'LEVELS': {'NAME': 0,
  'LABEL': 'Reduced',
  'DESCRIPTION': 'Reduced bicyling percentage of SOV tours less than 20 miles'}}

In [168]:
collect_scenario_info(cat_levels)[0]

{'NAME': 'B',
 'LABEL': 'Bicycles',
 'DESCRIPTION': 'Network improvements, incentives, and technologies that encourage bicycling and other light-weight vehicle travel',
 'INSTRUCTIONS': 'The diversion of single-occupant vehicle travel to bicycles, electric bicycles and other light-weight vehicles',
 'LEVELS': {'NAME': 0,
  'LABEL': 'Reduced',
  'DESCRIPTION': 'Reduced bicyling percentage of SOV tours less than 20 miles'}}

In [172]:
output3 = collect_scenario_info(cat_levels)

In [173]:
with open(f'{inpath}/VEScenarioViewer/test_out3.json', 'w') as f:
    json.dump(output3, f)